In [601]:
import json
import javalang
import numpy as np

data = open("./dataset/full_set.json", "r")

data_dict = json.load(data)
# data_dict

In [602]:
# data_dict[0]

In [603]:
def read_source(src):
    return src.split("\n")

def view_source(src):
    newSrc = src.splitlines()
    for s_unit in newSrc:
        print(s_unit)

In [604]:
# test_sample = data_dict[1]['source'] 
# view_source(test_sample)

In [605]:
# read_source(test_sample)

In [606]:
# tokens = list(javalang.tokenizer.tokenize(test_sample))

# for token in tokens:
#     print(f"Token: '{token.value}', position: {token.position}, is a {type(token)}")

In [607]:
# for token in tokens:
#     if isinstance(token, javalang.tokenizer.Keyword):
#         print(token)

In [608]:
"""
    Get the unique values and remove '}' character
"""
def unique(ending_s_units):
    ending_s_units = [ending_s_unit for ending_s_unit in ending_s_units if ending_s_unit != '}']
    return np.unique(np.array(ending_s_units))

"""
    Returns type of the method
"""
def get_method_type(src):
    # Add fake class in order to avoid error :D
    code_snippet = 'class A { \n' + src + '\n}'
    tree = javalang.parse.parse(code_snippet)
    for _, node in tree.filter(javalang.tree.MethodDeclaration):
        if node.return_type is None:
            return 'void'
        else:
            return node.return_type.name

"""
    Get the list of keywords
"""
def get_keywords(src):
   tokens = list(javalang.tokenizer.tokenize(src))
   return [token.value for token in tokens if isinstance(token, javalang.tokenizer.Keyword)]

"""
    Returns the Ending S_units.
"""
def get_ending_s_units(src):
    # tokens = list(javalang.tokenizer.tokenize(src))
    # keywords = get_keywords(src)
    s_units = read_source(src)
    num_of_lines = len(s_units)
    method_type = get_method_type(src)
    
    # For non-void return type
    inner_ending_s_units = []
    
    # For void return type
    outer_ending_s_unit = ""
    
    match method_type:
        case 'void':
            for i in range(num_of_lines):
                if 'if' or 'else' or 'else if' or 'try' or 'catch' or 'finally' or 'switch' in s_units[i]:
                    for j in range(i, num_of_lines):
                        if '}' in s_units[j]:
                            inner_ending_s_units.append(s_units[j - 1].strip())
                            break
            for k in reversed(range(num_of_lines)):
                if '}' not in s_units[k]:
                    outer_ending_s_unit = s_units[k].strip()
                    break
        case _:
            for i in range(num_of_lines):
                if 'return' in s_units[i]:
                    inner_ending_s_units.append(s_units[i].strip())
        
    if method_type == 'void':
        for inner in inner_ending_s_units:
            if outer_ending_s_unit == inner:
                return outer_ending_s_unit
        
    return unique(inner_ending_s_units)      

# Test sample
src = data_dict[25]['source']

# Get method type
# print(get_method_type(src))

# Get keywords
# print(get_keywords(src))

# Get Ending S Units
print(get_ending_s_units(src))

['return x;']
